# Following the article for Pose Estimation of [StackAbuse](https://stackabuse.com/pose-estimation-and-keypoint-detection-with-yolov7-in-python/)

In [1]:
! git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 1094, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1094 (delta 0), reused 1 (delta 0), pack-reused 1091
Receiving objects: 100% (1094/1094), 69.85 MiB | 4.08 MiB/s, done.
Resolving deltas: 100% (522/522), done.


In [1]:
%cd yolov7
!ls

/Users/johanneseder711/Documents/Python/01_active/Keepercam/01_pose_estimation/yolov7
LICENSE.md            figure                scripts
README.md             frames_pose_estimated test.py
cfg                   hubconf.py            tools
data                  inference             train.py
deploy                models                train_aux.py
detect.py             paper                 utils
export.py             requirements.txt      yolov7-w6-pose.pt


Whenever you run code with a given set of weights - they'll be downloaded and stored in this directory. To perform pose estimation, we'll want to download the weights for the pre-trained YOLOv7 model for that task, which can be found under the /releases/download/ tab on GitHub:

In [4]:
# Download the weights of the pretrained Yolo7 model to do pose estimation with it
! curl -L https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt -o yolov7-w6-pose.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  153M  100  153M    0     0  4369k      0  0:00:36  0:00:36 --:--:-- 4468k     0  0:00:36  0:00:23  0:00:13 4419k328k      0  0:00:36  0:00:25  0:00:11 4499k  0     0  4354k      0  0:00:36  0:00:31  0:00:05 4484k


# Loading the YOLOv7 Pose Estimation Model



Just realised that the article from StackAbuse is using Pytorch, which I am not familiar at all. I am more or less familiar with TensorFlow, but it seems that in Pytorch the Yolo7 model is already included, so they use it from there out of the box. 
Questions: 
* How do I install Pytorch
* Does it already support Apple Silicon (I want to use my M1 GPU)

## Installing Pytorch

Following this [Medium Article](https://towardsdatascience.com/installing-pytorch-on-apple-m1-chip-with-gpu-acceleration-3351dc44d67c) this could be straight forward. Let's follow it and see how this easy this turns out to be in pratice. 😅

In [2]:
# checking python version
! python --version

Python 3.9.5


⚠️ Version of Python needs to be 3.8 or 3.9!

In [2]:
! conda install -y pytorch

Solving environment: done

# All requested packages already installed.



In [3]:
! conda install -y torchvision 

Solving environment: done

# All requested packages already installed.



In [4]:
! conda install -c pytorch torchaudio -y

Solving environment: done

# All requested packages already installed.



Found that using conda the installation did work but I could not import the module from pytorch! Always got the error: "No module named torch!
Try installation via pip directly

In [7]:
! pip install torchvision 

  Using cached torchvision-0.14.0-cp39-cp39-macosx_11_0_arm64.whl (1.3 MB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 4.4 MB/s eta 0:00:0000:0100:01


using installation through pip works - found it on [StackOverflow](https://stackoverflow.com/questions/54843067/no-module-named-torch)!

In [3]:
import torch
from torchvision import transforms

/Users/johanneseder711/miniforge3/envs/venv_keepercam/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
! pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 2.3 MB/s eta 0:00:00


In [23]:
! pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 4.6 MB/s eta 0:00:0000:0100:01


In [ ]:
! pip install pyyaml

In [32]:
! pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 KB 3.4 MB/s eta 0:00:0000:0100:01


In [34]:
! pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 4.4 MB/s eta 0:00:0000:0100:01


To finally be able to import all libraries below, I needed to install a lot of additional libraries (see above)! They will show up below as import error if you do not have them installed in your environment

In [4]:
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

import matplotlib.pyplot as plt
import cv2
import numpy as np

Now, we will check if PyTorch can find the Metal Performance Shaders plugin.

In [5]:
import torch
import math
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

True
True


## 😫 Using GPU is not working yet, so I will go with CPU to try it first

In [21]:
def load_model():
    #device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    device = torch.device("cpu")
    model = torch.load('yolov7-w6-pose.pt', map_location=device)['model']
    # Put in inference mode
    model.float().eval()
    return model
    
    if torch.backends.mps.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)


In [22]:

model = load_model()

In [17]:
def run_inference(url, model):
    image = cv2.imread(url) # shape: (480, 640, 3)
    # Resize and pad image
    image = letterbox(image, 960, stride=64, auto=True)[0] # shape: (768, 960, 3)
    # Apply transforms
    image = transforms.ToTensor()(image) # torch.Size([3, 768, 960])
    # Turn image into batch
    image = image.unsqueeze(0) # torch.Size([1, 3, 768, 960])
    output, _ = model(image) # torch.Size([1, 45900, 57])
    return output, image

In [18]:
def save_pose_estimated_frame(output, image, path):
    global dirname
    output = non_max_suppression_kpt(output, 
                                     0.25, # Confidence Threshold
                                     0.65, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)
    with torch.no_grad():
        output = output_to_keypoint(output)
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    for idx in range(output.shape[0]):
        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(nimg)
    plt.savefig(dirname+'/'+ path.stem +'_pose_estimated')
    plt.close()

In [108]:
dirname = "frames_pose_estimated"
! mkdir "frames_pose_estimated"

In [9]:
import os
from pathlib import Path

In [10]:
path_to_dir = Path('/Users/johanneseder711/Documents/Python/01_active/Keepercam/00_extract_frames/extracted_frames/')

In [19]:
for obj in path_to_dir.glob('*'):
    if obj.is_file and obj.exists():
        output, image = run_inference(str(obj), model)
        save_pose_estimated_frame(output, image, obj)

/Users/johanneseder711/miniforge3/envs/venv_keepercam/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/Users/johanneseder711/Documents/Python/01_active/Keepercam/01_pose_estimation/yolov7/utils/general.py:728: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  x = x[xc[xi]]  # confidence


NotImplementedError: The operator 'torchvision::nms' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.